In [1]:
import sys
import json
import shutil
import os
import glob
import pandas as pd
from IPython.display import Image, display

In [2]:
crimes = pd.read_csv('../data/cleaned/crime-processed.csv')
crimes.head()

,DR_NO,Date Rptd,Year,AREA NAME,Rpt Dist No,Crime Type,Crime Severity,Crm Cd Desc,Arrested,LAT,LON
0,1307355,2010-02-20,2010,Newton,1385,Statutory,Infraction,VIOLATION OF COURT ORDER,1,33.9825,-118.2695
1,11401303,2010-09-13,2010,Pacific,1485,Property,Felony,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0,33.9599,-118.3962
2,70309629,2010-08-09,2010,Newton,1324,Financial/Other,Others,OTHER MISCELLANEOUS CRIME,0,34.0224,-118.2524
3,90631215,2010-01-05,2010,Hollywood,646,Statutory,Infraction,VIOLATION OF COURT ORDER,0,34.1016,-118.3295
4,100100501,2010-01-03,2010,Central,176,Inchoate,Felony,"RAPE, ATTEMPTED",0,34.0387,-118.2488


In [ ]:
crimes.isnull().mean()

In [ ]:
crimes.dtypes

In [ ]:
display(Image('../viz/EDA/Crime/crimes_by_year.png'))

In [ ]:
display(Image('../viz/EDA/Crime/crimes_by_area.png'))

In [ ]:
display(Image('../viz/EDA/Crime/crimes_by_area_year.png'))

In [ ]:
display(Image('../viz/EDA/Crime/arrests_by_year.png'))

In [ ]:
display(Image('../viz/EDA/Crime/arrests_by_area_year.png'))

In [ ]:
display(Image('../viz/EDA/Crime/tp_distr.png'))

In [ ]:
display(Image('../viz/EDA/Crime/tp_distr_area.png'))

In [ ]:
display(Image('../viz/EDA/Crime/tp_distr_by_area_year.png'))

In [ ]:
display(Image('../viz/EDA/Crime/sev_distr.png'))

In [ ]:
display(Image('../viz/EDA/Crime/sev_distr_by_area.png'))

In [ ]:
display(Image('../viz/EDA/Crime/sev_distr_by_area_year.png'))